# 🎯 CaseGenius Enhanced: AI Case Study Generator with Ambiguity Resolution

**New Feature:** Intelligent ambiguity detection and interactive clarification

**Flow:**
1. Enter case brief
2. AI detects ambiguities
3. Configure which questions to ask (max 3)
4. Answer questions
5. Generate comprehensive case study

**Time:** 10-15 minutes | **Cost:** ~$4-5

---

In [11]:

import subprocess, json
import os, subprocess, json, time, re
from typing import List, Dict, Tuple


# ---------------------------
# Local configuration (NO env variables)
# ---------------------------
LLM_MODE: str = 'gemini'                 # 'mock' | 'ollama' | 'gemini'
OLLAMA_MODEL: str = 'llama3'             # local model name for Ollama
GEMINI_API_KEY: str = 'API_KEY'   # <-- put your API key here
GEMINI_MODEL: str = 'gemini-2.5-flash'   # recommended fast, balanced model

# Toggle: use the official SDK (google.generativeai) or fall back to raw REST
USE_GOOGLE_GENERATIVEAI_SDK: bool = True

class LLMAdapter:
    def __init__(self, mode: str):
        self.mode = mode

    def generate(self, prompt: str) -> str:
        if self.mode == 'mock':
            return '[MOCK ANSWER] ' + prompt[:300]

        if self.mode == 'ollama':
            try:
                r = subprocess.run(
                    ['ollama', 'run', OLLAMA_MODEL, prompt],
                    capture_output=True, text=True
                )
                return r.stdout.strip() or r.stderr.strip() or '[OLLAMA EMPTY]'
            except Exception as e:
                return '[OLLAMA ERROR] ' + str(e)

        if self.mode == 'gemini':
            # --- Preferred: google.generativeai SDK (works well in Colab/Jupyter) ---
            if USE_GOOGLE_GENERATIVEAI_SDK:
                try:
                    import google.generativeai as genai
                    genai.configure(api_key=GEMINI_API_KEY)   # explicit local key
                    model = genai.GenerativeModel(GEMINI_MODEL)
                    response = model.generate_content(prompt)
                    return getattr(response, 'text', None) or '[GEMINI EMPTY]'
                except Exception as sdk_err:
                    # Fall back to REST if SDK not installed or import fails
                    try:
                        import requests
                        url = f"https://generativelanguage.googleapis.com/v1beta/models/{GEMINI_MODEL}:generateContent?key={GEMINI_API_KEY}"
                        payload = {
                            "contents": [
                                {"role": "user", "parts": [{"text": prompt}]}
                            ]
                        }
                        r = requests.post(url, json=payload, timeout=30)
                        r.raise_for_status()
                        data = r.json()
                        try:
                            return data["candidates"][0]["content"]["parts"][0].get("text", "") or json.dumps(data)[:800]
                        except Exception:
                            return json.dumps(data)[:800]
                    except Exception as rest_err:
                        return f"[GEMINI REST ERROR] {rest_err} | [SDK ERROR] {sdk_err}"

            # --- Alternative: Raw REST only ---
            else:
                try:
                    import requests
                    if not GEMINI_API_KEY or GEMINI_API_KEY == 'PASTE_YOUR_KEY':
                        return '[GEMINI CONFIG ERROR] Provide a valid GEMINI_API_KEY (local variable)'

                    url = f"https://generativelanguage.googleapis.com/v1beta/models/{GEMINI_MODEL}:generateContent?key={GEMINI_API_KEY}"
                    payload = {
                        "contents": [
                            {"role": "user", "parts": [{"text": prompt}]}
                        ]
                    }
                    resp = requests.post(url, json=payload, timeout=30)
                    resp.raise_for_status()
                    data = resp.json()
                    try:
                        return data["candidates"][0]["content"]["parts"][0].get("text", "") or json.dumps(data)[:800]
                    except Exception:
                        return json.dumps(data)[:800]
                except Exception as e:
                    return '[GEMINI REST ERROR] ' + str(e)

        return '[UNKNOWN MODE]'

# ---------------------------
# Usage in Jupyter
# ---------------------------
llm = LLMAdapter(LLM_MODE)
print('LLM_MODE =', LLM_MODE)
print(llm.generate("Explain how AI works in a few words"))

LLM_MODE = gemini
Learns from data patterns to predict or decide.


In [12]:
CASE_BRIEF = """
Create a comprehensive case study for a sub-brand launch.

Company: L'Oréal

Analysis Required:
1. CapEx & OpEx comparison with competitors
2. Market modeling and sizing
3. P&L projections
4. Business Canvas
5. Competitor Analysis
6. Market gap identification

Timeline: Launch in the next fiscal year.
Target market: High-growth segments.
"""

print("📋 Case brief loaded")


📋 Case brief loaded


In [13]:
class AmbiguityDetector:
    def detect(self, brief: str):
        print("🔍 Detecting ambiguities...")

        prompt = f"""
Analyze the following case brief and identify ambiguities.

Return ONLY a JSON array.
Each item must have:
- term
- category
- question
- options (3–4)
- importance (HIGH/MEDIUM/LOW)

Case Brief:
{brief}
"""

        try:
            response = llm.generate(prompt)
            match = re.search(r"\[.*\]", response, re.DOTALL)
            return json.loads(match.group()) if match else []
        except Exception as e:
            print("❌ Ambiguity detection failed:", e)
            return []


In [14]:
detector = AmbiguityDetector()
detected_ambiguities = detector.detect(CASE_BRIEF)

print(f"\nFound {len(detected_ambiguities)} ambiguities:\n")
for i, amb in enumerate(detected_ambiguities, 1):
    print(f"{i}. [{amb['importance']}] {amb['term']}")

🔍 Detecting ambiguities...

Found 11 ambiguities:

1. [HIGH] Sub-brand definition
2. [MEDIUM] Comprehensive case study deliverables
3. [HIGH] Competitors for CapEx & OpEx
4. [MEDIUM] CapEx & OpEx comparison detail
5. [HIGH] Market for modeling and sizing
6. [HIGH] P&L projection details
7. [LOW] Business Canvas format
8. [HIGH] Competitors for Competitor Analysis
9. [MEDIUM] Market gap criteria
10. [HIGH] Launch definition and timeline
11. [HIGH] High-growth segments


In [15]:
detector = AmbiguityDetector()
detected_ambiguities = detector.detect(CASE_BRIEF)

print(f"\nFound {len(detected_ambiguities)} ambiguities:\n")
for i, amb in enumerate(detected_ambiguities, 1):
    print(f"{i}. [{amb['importance']}] {amb['term']}")


🔍 Detecting ambiguities...

Found 9 ambiguities:

1. [HIGH] sub-brand launch
2. [HIGH] comprehensive case study
3. [HIGH] competitors (CapEx & OpEx comparison)
4. [MEDIUM] CapEx & OpEx comparison
5. [MEDIUM] P&L projections
6. [HIGH] next fiscal year
7. [HIGH] High-growth segments
8. [MEDIUM] Market gap identification
9. [MEDIUM] Competitor Analysis (point 5)


In [16]:
QUESTIONS_TO_ASK = [
    i for i, amb in enumerate(detected_ambiguities)
    if amb["importance"] == "HIGH"
][:3]

print("\nQuestions selected:")
for idx in QUESTIONS_TO_ASK:
    print("-", detected_ambiguities[idx]["term"])



Questions selected:
- sub-brand launch
- comprehensive case study
- competitors (CapEx & OpEx comparison)


In [17]:
user_answers = {}

for idx in QUESTIONS_TO_ASK:
    amb = detected_ambiguities[idx]

    print("\n❓", amb["question"])
    for i, opt in enumerate(amb["options"], 1):
        print(f"{i}. {opt}")

    while True:
        choice = input("Your choice: ").strip()
        if choice.isdigit() and 1 <= int(choice) <= len(amb["options"]):
            user_answers[amb["term"]] = amb["options"][int(choice)-1]
            break



❓ What is the nature of the 'sub-brand launch'?
1. A completely new brand under the L'Oréal corporate umbrella.
2. A new product line or extension under an existing L'Oréal major brand (e.g., L'Oréal Paris, Lancôme).
3. A strategic initiative rebranding an acquired small brand.
Your choice: 1

❓ What is the primary purpose and expected output of this 'comprehensive case study'?
1. A detailed strategic business plan for the sub-brand's launch.
2. A pre-mortem analysis identifying potential risks and opportunities for the launch.
3. A strategic document outlining key decisions and assumptions for the launch.
4. An academic-style analysis of a hypothetical sub-brand launch.
Your choice: 1

❓ Which competitors should be included in the CapEx & OpEx comparison?
1. Direct competitors within the specific high-growth segment identified for the sub-brand.
2. Major beauty industry competitors of L'Oréal (e.g., Estée Lauder, Unilever, P&G).
3. Both direct sub-brand competitors and major beauty i

In [18]:
def enrich_brief(brief: str, clarifications: Dict[str, str]) -> str:
    if not clarifications:
        return brief

    enriched = brief + "\n\nCLARIFICATIONS:\n"
    for k, v in clarifications.items():
        enriched += f"- {k}: {v}\n"
    return enriched

ENRICHED_BRIEF = enrich_brief(CASE_BRIEF, user_answers)
print("✅ Brief enriched")


✅ Brief enriched


In [19]:
class Agent:
    def __init__(self, name: str, role: str):
        self.name = name
        self.role = role

    def run(self, brief: str, context: str = "") -> str:
        print(f"🔄 Running {self.name}...")

        prompt = f"""
You are a {self.role}.

Generate a professional {self.name}.

Case Brief:
{brief}

Previous Context:
{context}
"""

        return llm.generate(prompt)


In [ ]:
agents = [
    Agent("Executive Summary", "Senior Strategy Consultant"),
    Agent("CapEx & OpEx Analysis", "Financial Analyst"),
    Agent("Market Modeling", "Market Research Analyst"),
]

results = {}
context = ""

for agent in agents:
    output = agent.run(ENRICHED_BRIEF, context)
    results[agent.name] = output
    context += output[:500]


🔄 Running Executive Summary...
🔄 Running CapEx & OpEx Analysis...


In [ ]:
output_dir = f"case_study_{time.strftime('%Y%m%d_%H%M%S')}"
os.makedirs(output_dir, exist_ok=True)

for name, text in results.items():
    filename = name.replace(" ", "_") + ".md"
    with open(os.path.join(output_dir, filename), "w", encoding="utf-8") as f:
        f.write(text)

with open(os.path.join(output_dir, "clarifications.json"), "w") as f:
    json.dump(user_answers, f, indent=2)

print("📁 Saved results to:", output_dir)


In [ ]:
print("\nSUMMARY")
print("Ambiguities detected:", len(detected_ambiguities))
print("Clarifications answered:", len(user_answers))
print("Documents generated:", len(results))

for k in results:
    print("-", k)
